In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import mne
import os
import re
import torch
import pickle

from utils import *

In [2]:
task_dict = {
    'Resting state': {'annotation': 'T0', 'runs': ['03', '04', '05', '06','07', '08', '09', '10', '11', '12', '13', '14']},
    'Left fist, performed': {'annotation': 'T1', 'runs': ['03', '07' '11']},
    'Left fist, imagined': {'annotation': 'T1', 'runs': ['04', '08', '12']},
    'Both fists, performed': {'annotation': 'T1', 'runs': ['05', '09', '13']},
    'Both fists, imagined': {'annotation': 'T1', 'runs': ['06', '10', '14']},
    'Right fist, performed': {'annotation': 'T2', 'runs': ['03', '07', '11']},
    'Right fist, imagined': {'annotation': 'T2', 'runs': ['04', '08', '12']},
    'Both feet, performed': {'annotation': 'T2', 'runs': ['05', '09', '13']},
    'Both feet, imagined': {'annotation': 'T2', 'runs': ['06', '10', '14']}
}


label_dict = {
    'T0': {'03': 'Resting state', '04': 'Resting state', '05': 'Resting state', '06': 'Resting state', '07': 'Resting state', '08': 'Resting state', '09': 'Resting state', '10': 'Resting state', '11': 'Resting state', '12': 'Resting state', '13': 'Resting state', '14': 'Resting state'},
    'T1': {'03': 'Left fist, performed', '04': 'Left fist, imagined', '05': 'Both fists, performed', '06': 'Both fists, imagined', '07': 'Left fist, performed', '08': 'Left fist, imagined', '09': 'Both fists, performed', '10': 'Both fists, imagined', '11': 'Left fist, performed', '12': 'Left fist, imagined', '13': 'Both fists, performed', '14': 'Both fists, imagined'},
    'T2': {'03': 'Right fist, performed', '04': 'Right fist, imagined', '05': 'Both feet, performed', '06': 'Both feet, imagined', '07': 'Right fist, performed', '08': 'Right fist, imagined', '09': 'Both feet, performed', '10': 'Both feet, imagined', '11': 'Right fist, performed', '12': 'Right fist, imagined', '13': 'Both feet, performed', '14': 'Both feet, imagined'}
}


In [3]:
DATA_PATH_RAW = '/home/williamtheodor/Documents/DL for EEG Classification/data/eegmmidb (raw)/files/'
DATA_PATH_LABELLED = '/home/williamtheodor/Documents/DL for EEG Classification/data/eegmmidb (labelled)/'


# create folder for each label
for label in task_dict.keys():
    if not os.path.exists(DATA_PATH_LABELLED + label):
        os.makedirs(DATA_PATH_LABELLED + label)

In [23]:
# renames TUH channels to conventional 10-20 system
def TUH_rename_ch(MNE_raw):
    for channel_name in MNE_raw.info["ch_names"]:
        reSTR = r"(?<=EEG )(.*)(?=-)" # working reSTR = r"(?<=EEG )(.*)(?=-REF)"
        reLowC = ['FP1', 'FP2', 'FZ', 'CZ', 'PZ']

        if re.search(reSTR, channel_name) and re.search(reSTR, channel_name).group() in reLowC:
            lowC = channel_name[0:5]+channel_name[5].lower()+channel_name[6:]
            mne.channels.rename_channels(MNE_raw.info, {channel_name: re.findall(reSTR, lowC)[0]})
        elif channel_name == "PHOTIC-REF":
            mne.channels.rename_channels(MNE_raw.info, {channel_name: "PHOTIC"})
        elif re.search(reSTR, channel_name):
            mne.channels.rename_channels(MNE_raw.info, {channel_name: re.findall(reSTR, channel_name)[0]})
        else:
            continue

    if 'T7' in MNE_raw.info["ch_names"] and 'T8' in MNE_raw.info["ch_names"] and 'P7' in MNE_raw.info["ch_names"] and 'P8' in MNE_raw.info["ch_names"]:
        mne.channels.rename_channels(MNE_raw.info, {'T7': 'T3', 'T8': 'T4', 'P7': 'T5', 'P8': 'T6'})

    TUH_pick = ['Fp1','Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Cz']  # A1, A2 removed
    
    MNE_raw.pick_channels(ch_names=TUH_pick)
    MNE_raw.reorder_channels(TUH_pick)

    return MNE_raw

In [22]:
file_path = DATA_PATH_RAW + 'S001/S001R03.edf'
raw = get_raw(file_path)

# rename channels
raw = TUH_rename_ch(raw)
raw.info['ch_names']

hey


['Fp1',
 'Fp2',
 'F3',
 'F4',
 'C3',
 'C4',
 'P3',
 'P4',
 'O1',
 'O2',
 'F7',
 'F8',
 'T3',
 'T4',
 'T5',
 'T6',
 'Cz']

In [28]:
NUMBER_SAMPLES = 1024
NUMBER_CHANNELS = 17

for patient_folder in tqdm(os.listdir(DATA_PATH_RAW)):
    for file in os.listdir(DATA_PATH_RAW + patient_folder + '/'):
        if file.endswith('.edf') and file[5:7] not in ['01', '02']:
            file_path = DATA_PATH_RAW + patient_folder + '/' + file

            raw = get_raw(file_path)
            annotations = get_annotations(file_path)

            annotation_dict = get_window_dict(raw, annotations)

            for annotation in annotation_dict.keys():
                for raw in annotation_dict[annotation]:
                    raw = TUH_rename_ch(raw.copy())

                    try: 
                        x = torch.tensor(raw.copy().get_data()[:,:NUMBER_SAMPLES].reshape(1,NUMBER_CHANNELS,NUMBER_SAMPLES))
                        label = label_dict[annotation][file[5:7]]
                        
                        picklePath = DATA_PATH_LABELLED + label + '/' + file[0:7] + '_' + annotation + '_' + label + '.pkl'
                        with open(picklePath, 'wb') as handle:
                            pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)
                    except:
                        print('Error with file: ' + file, raw.copy().get_data().shape)
                    

  4%|▎         | 4/109 [00:22<09:50,  5.62s/it]

Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R10.edf (17, 353)
Error with file: S088R14.edf (17, 353)
Error with file: S088R14.edf (17, 353)
Error with file: S088R14.edf (17, 353)
Error with file: S088R14.edf (17, 353)
Error with file: S088R14.edf (17, 353)
Error with file: S088R14.edf (17, 353)
Error with file: S088R14.

  5%|▍         | 5/109 [00:27<09:32,  5.51s/it]

Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)
Error with file: S088R11.edf (17, 353)


 46%|████▌     | 50/109 [04:42<05:31,  5.62s/it]/home/williamtheodor/Documents/DL for EEG Classification/BENDR-XAI/notebooks/utils.py:27: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_file_path, verbose=False, preload=True)
/home/williamtheodor/Documents/DL for EEG Classification/BENDR-XAI/notebooks/utils.py:27: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_file_path, verbose=False, preload=True)
/home/williamtheodor/Documents/DL for EEG Classification/BENDR-XAI/notebooks/utils.py:27: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_file_path, verbose=False, preload=True)
/home/williamtheodor/Documents/DL for EEG Classification/BENDR-XAI/notebooks/utils.py:27: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_file_pat

...

 71%|███████   | 77/109 [07:13<03:00,  5.63s/it]

Error with file: S106R05.edf (17, 795)


 82%|████████▏ | 89/109 [08:19<01:50,  5.52s/it]

Error with file: S104R08.edf (17, 763)


100%|██████████| 109/109 [10:11<00:00,  5.61s/it]


In [ ]:
raw.info['ch_names']